<div style="display: flex; justify-content: center; margin-bottom: 20px;">
  <img src="../docs/_static/seispolarity_logo_title.svg">
</div>

---

## Model API Usage Example


Import necessary libraries

In [1]:
# Import necessary libraries for model operations
import numpy as np
import torch
import obspy
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent))
from seispolarity.models import SCSN


## Ross (SCSN) Model

SCSN (Ross) is a polarity prediction model with the following characteristics:

- **Input length**: 400 samples
- **Output classes**: 3 (Up/Down/Unknown)
- **Architecture**: Shared backbone network with convolutional layers

The model classifies seismic waveforms into three polarity categories:
- **Up**: Positive polarity
- **Down**: Negative polarity  
- **Unknown**: Undecidable polarity

## Creating a Ross (SCSN) Model

In [2]:
# Create a Ross (SCSN) model
# num_fm_classes=3: number of polarity classes (Up, Down, Unknown)
# sample_rate=100: expected sampling rate in Hz
ross_model = SCSN(
    num_fm_classes=3,
    sample_rate=100.0
)

print(f"Ross (SCSN) model created successfully!")
print(f"Model name: {ross_model.name}")
print(f"Device: {ross_model.device}")
print(f"\nModel architecture:")
print(ross_model)

Ross (SCSN) model created successfully!
Model name: SCSN
Device: cuda

Model architecture:
SCSN(
  (shared_backbone): SharedBackbone(
    (sequential): Sequential(
      (0): Conv1d(1, 32, kernel_size=(21,), stride=(1,), padding=same)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv1d(32, 64, kernel_size=(15,), stride=(1,), padding=same)
      (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv1d(64, 128, kernel_size=(11,), stride=(1,), padding=same)
      (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (12): Flatten(start_dim=1,

## Using the Model with PyTorch Tensors

In [4]:
# Create random input tensor
# Shape: (batch_size, channels, length)
# SCSN expects 400-point input waveforms
batch_size = 512
channels = 1
length = 400

random_input = torch.randn(batch_size, channels, length)
print(f"Input tensor shape: {random_input.shape}")

# Apply the model
ross_model.eval()
with torch.no_grad():
    output = ross_model(random_input)

print(f"\nOutput tensor shape: {output.shape}")
print(f"Output (logits):\n{output}")

# Convert to probabilities
probs = torch.softmax(output, dim=1)
print(f"\nProbabilities:\n{probs}")

# Get predictions (0=Down, 1=Up, 2=Unknown)
predictions = torch.argmax(probs, dim=1)
print(f"\nPredictions: {predictions}")

# Map predictions to polarity labels
label_map = {0: "Down", 1: "Up", 2: "Unknown"}
for i, pred in enumerate(predictions):
    confidence = probs[i, pred].item()
    label = label_map[pred.item()]
    print(f"  Sample {i+1}: {label} (confidence={confidence:.4f})")

Input tensor shape: torch.Size([512, 1, 400])

Output tensor shape: torch.Size([512, 3])
Output (logits):
tensor([[ 0.0359, -0.0132,  0.0075],
        [ 0.0229, -0.0204,  0.0061],
        [ 0.0362, -0.0143, -0.0009],
        ...,
        [ 0.0212, -0.0144,  0.0018],
        [ 0.0326, -0.0115,  0.0025],
        [ 0.0266, -0.0159, -0.0004]])

Probabilities:
tensor([[0.3420, 0.3256, 0.3324],
        [0.3400, 0.3256, 0.3344],
        [0.3431, 0.3262, 0.3306],
        ...,
        [0.3395, 0.3276, 0.3329],
        [0.3416, 0.3269, 0.3315],
        [0.3411, 0.3269, 0.3320]])

Predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,